In [1]:
import pandas as pd 
import pandas_profiling as pp 
aguadataframe = pd.read_csv('Calidad_de_agua_2022.csv',delimiter = ';')
aguadataframe
aguadataframe.info()

c:\Users\ramir\Fundamentos\Practicoespecial\PracticoEspecialGrupo14\practicoEspecial\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   orden                    168 non-null    int64 
 1   sitios                   168 non-null    object
 2   codigo                   168 non-null    object
 3   fecha                    168 non-null    object
 4   año                      168 non-null    object
 5   campaña                  168 non-null    object
 6   tem_agua                 168 non-null    object
 7   tem_aire                 166 non-null    object
 8   od                       168 non-null    object
 9   ph                       168 non-null    object
 10  olores                   168 non-null    object
 11  color                    168 non-null    object
 12  espumas                  168 non-null    object
 13  mat_susp                 168 non-null    object
 14  colif_fecales_ufc_100ml  167 non-null    o

C:\Users\ramir\AppData\Local\Temp\ipykernel_9880\1932533977.py:2: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling as pp


viendo la informacion del dataset, se puede ver que hay algunas columnas que contienen valores nulos (informacion para un posterior analisis) y ademas que todas las columnas son de tipo objet lo cual nos da un indicio de que algunas columnas vamos a tener que transformarlas y cambiarlas de tipo. 

In [2]:
aguadataframe = aguadataframe.drop(columns='orden')

tomamos la decision de eliminar la columna orden debido a que no aportaba ningun tipo de informacion para el analisis de los datos. 

In [3]:
aguadataframe.columns
aguadataframe.describe()

,sitios,codigo,fecha,año,campaña,tem_agua,tem_aire,od,ph,olores,...,dbo_mg_l,dqo_mg_l,turbiedad_ntu,hidr_deriv_petr_ug_l,cr_total_mg_l,cd_total_mg_l,clorofila_a_ug_l,microcistina_ug_l,ica,calidad_de_agua
count,168,168,168,168,168,168,166,168,168,168,...,167,168,168,168,168,168,167,168,155,154
unique,42,42,7,3,6,93,32,130,116,6,...,55,51,63,6,27,4,70,15,39,3
top,Canal Villanueva y Río Luján,TI001,23/2/2022,2022,Verano,no se midió,14,no se midió,no se midió,Ausencia,...,no se midió,<30,no se midió,<0.10,<0.005,<0.001,<10,<0.15,40,Extremadamente deteriorada
freq,4,4,42,163,42,19,25,24,23,120,...,53,51,10,113,113,147,74,76,13,92


como vemos cuando hacemos un describe nos muestra pocos datos y omite los estadisticos como la media, los cuartiles, etc. Esto es debido a que todas las variables son de tipo object asique lo que habria que hacer es convertirlas a algun otro tipo para poder analizarlas. 

In [4]:
aguadataframe1= aguadataframe.replace("no se midio",pd.NA)
aguadataframe1 = aguadataframe.replace({"<0.10": 0.10, "<0.005": 0.005, "<0.001": 0.001, "<0.15": 0.15, "<30": 30})
columnas_numericas=['tem_agua', 'tem_aire', 'od', 'ph', 'dbo_mg_l', 'dqo_mg_l', 
                      'turbiedad_ntu', 'hidr_deriv_petr_ug_l', 'cr_total_mg_l', 
                      'cd_total_mg_l', 'clorofila_a_ug_l', 'microcistina_ug_l', 'ica','colif_fecales_ufc_100ml', 'escher_coli_ufc_100ml', 
                      'enteroc_ufc_100ml', 'nitrato_mg_l', 'nh4_mg_l', 
                      'p_total_l_mg_l', 'fosf_ortofos_mg_l']
aguadataframe1[columnas_numericas]=aguadataframe[columnas_numericas].apply(pd.to_numeric, errors='coerce')
print(aguadataframe1.describe())


         tem_agua    tem_aire          od          ph  \
count  145.000000  144.000000  132.000000  112.000000   
mean    17.823655   15.793056    6.747348    7.570000   
std      4.848688    5.124351    2.835764    0.770363   
min      6.000000    4.000000    0.360000    5.000000   
25%     14.600000   13.000000    5.065000    7.075000   
50%     17.900000   14.000000    6.650000    7.485000   
75%     20.400000   19.000000    8.580000    8.002500   
max     27.400000   27.000000   17.610000   10.020000   

       colif_fecales_ufc_100ml  escher_coli_ufc_100ml  enteroc_ufc_100ml  \
count             1.530000e+02             153.000000         153.000000   
mean              8.669023e+04            4093.392157         951.705882   
std               3.812712e+05           15058.086505        3008.483046   
min               8.000000e+01               1.000000           2.000000   
25%               1.200000e+03             100.000000          50.000000   
50%               4.000000e+03

ahora que transformamos los valores se puede ver un mejor analisis del metodo describe, por ejemplo se puede ver la media de la temperatura del agua que es de 17.82, el desvio, los valores maximos y minimos y los cuartiles.  Para las columnas que tienen valores que dicen "ausencia", "presencia", las vamos a convertir en booleanas para por ejemplo analizar la frecuencia o analizar algun tipo de correlacion con alguna otra variable.

In [5]:
boolean_columns = ['espumas', 'olores', 'color', 'mat_susp']
for col in boolean_columns:
    aguadataframe1[col] = aguadataframe1[col].replace({'Presencia': True, 'Ausencia': False})
for col in boolean_columns:
    aguadataframe1[col] = aguadataframe1[col].astype(bool)
aguadataframe1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sitios                   168 non-null    object 
 1   codigo                   168 non-null    object 
 2   fecha                    168 non-null    object 
 3   año                      168 non-null    object 
 4   campaña                  168 non-null    object 
 5   tem_agua                 145 non-null    float64
 6   tem_aire                 144 non-null    float64
 7   od                       132 non-null    float64
 8   ph                       112 non-null    float64
 9   olores                   168 non-null    bool   
 10  color                    168 non-null    bool   
 11  espumas                  168 non-null    bool   
 12  mat_susp                 168 non-null    bool   
 13  colif_fecales_ufc_100ml  153 non-null    float64
 14  escher_coli_ufc_100ml    1

una vez que ya pasamos todas las variables a un tipo mas analizable, podemos hacer la matriz de correlación para ver si hay variables que se relacionar por tener una correlación alta, eso si no siempre que tengan una correlacion alta significa que las variables están relacionadas

In [8]:
numeric_columns= aguadataframe1.select_dtypes(include=['float64','int64'])
correlation_matrix=numeric_columns.corr()
correlation_matrix

,tem_agua,tem_aire,od,ph,colif_fecales_ufc_100ml,escher_coli_ufc_100ml,enteroc_ufc_100ml,nitrato_mg_l,nh4_mg_l,p_total_l_mg_l,fosf_ortofos_mg_l,dbo_mg_l,dqo_mg_l,turbiedad_ntu,hidr_deriv_petr_ug_l,cr_total_mg_l,cd_total_mg_l,clorofila_a_ug_l,microcistina_ug_l,ica
tem_agua,1.000000,0.749406,-0.262059,-0.205391,-0.141762,-0.177103,-0.105951,-0.270049,-0.283294,-0.098941,-0.136711,-0.210644,-0.267997,-0.056700,NaN,0.067740,NaN,-0.443524,-0.243485,0.270831
tem_aire,0.749406,1.000000,-0.174873,-0.130082,-0.057018,-0.083639,0.005996,-0.080295,-0.181666,-0.065497,-0.013985,-0.198447,-0.242875,-0.102224,NaN,-0.367683,NaN,-0.113665,-0.169679,0.045179
od,-0.262059,-0.174873,1.000000,0.658103,-0.230619,-0.108482,-0.037529,0.132093,-0.365970,-0.061781,-0.293592,0.072599,0.039675,0.202578,NaN,0.211046,NaN,0.107393,0.142799,0.250481
ph,-0.205391,-0.130082,0.658103,1.000000,-0.092894,0.007041,0.067109,0.101876,-0.065824,0.060005,0.152632,0.179704,0.082101,0.124389,NaN,0.075167,NaN,0.221591,0.451394,0.069065
colif_fecales_ufc_100ml,-0.141762,-0.057018,-0.230619,-0.092894,1.000000,0.057524,0.036243,0.053802,0.545473,0.056742,0.586666,0.717953,0.131823,-0.063592,NaN,-0.178125,NaN,0.109840,0.438363,-0.290986
escher_coli_ufc_100ml,-0.177103,-0.083639,-0.108482,0.007041,0.057524,1.000000,0.775455,0.025356,0.100978,0.327762,0.097340,-0.029728,0.065741,-0.082120,NaN,-0.165633,NaN,-0.102407,0.308775,-0.192756
enteroc_ufc_100ml,-0.105951,0.005996,-0.037529,0.067109,0.036243,0.775455,1.000000,-0.110905,0.043876,0.221840,0.211363,-0.019023,-0.026061,-0.098137,NaN,-0.054940,NaN,-0.119739,0.547485,-0.207331
nitrato_mg_l,-0.270049,-0.080295,0.132093,0.101876,0.053802,0.025356,-0.110905,1.000000,-0.130251,-0.076199,-0.019057,0.045032,0.211575,-0.125435,NaN,-0.272003,NaN,0.323598,0.743591,-0.248455
nh4_mg_l,-0.283294,-0.181666,-0.365970,-0.065824,0.545473,0.100978,0.043876,-0.130251,1.000000,0.036159,0.551613,0.745666,0.146103,-0.230782,NaN,-0.287375,NaN,0.217224,-0.219998,-0.382761
p_total_l_mg_l,-0.098941,-0.065497,-0.061781,0.060005,0.056742,0.327762,0.221840,-0.076199,0.036159,1.000000,0.072395,0.678648,-0.059295,-0.115121,NaN,-0.072907,NaN,-0.069197,-0.385399,-0.125223
